In [25]:
import numpy as np

import os

import pydicom

import itertools

import scipy.io

# Extract velocity encoding information


init_path = '/home/andres/Documents/_Data/CKD_Part1/'

dest_path = '/home/andres/Documents/_Data/venc_info/'

study = 'CKD1'

key = 'QFLOW'


def orientationExtractor(folder):

    """
    Extract whether files to be read belong to right or left kidney.

    Params:
        - inherited from class (see description at beginning)
        - folder: folder name where to check for information

    Return: orientation ('dx' if right, 'si' if left)

    """ 

    if ('dx' in folder) or ('DX' in folder) or ('Right' in folder):

        orientation = 'dx'

    elif ('si' in folder) or ('SI' in folder) or ('Left' in folder):

        orientation = 'si'

    return orientation





def vencFromDCM(key, study, init_path, dest_path):
    
    
    """
    Extracts velocity encoding information from DICOM files.
    
    Params:
    
    - key: string to identify QFLOW acquisitions
    
    - study: study from where images are extracted
    
    - init_path: folder from where to start looking for the DICOM files with the VENCs
    
    - dest_path: folder where to save TXT files with results
    
    Outputs:
    
    - Text file with list of files
    
    - Text file with list of VENCs acquired
    
    """
    
    # Check that the first path exists
    
    if os.path.isdir(init_path):
    
        vencs = []

        files = []

        patients = sorted(os.listdir(init_path)) # Patients from study

        for patient in patients: # Loop through all patients

            if 'MRI' in patient:

                patientID = study + '_' + patient[:11]

                patient_path = init_path + patient + '/'

                orientations = sorted(os.listdir(patient_path)) # Folders with orientation info

                for orientation in orientations:

                    if 'QFLOW' in orientation:

                        orient = orientationExtractor(orientation)

                        orient_path = patient_path + orientation + '/'

                        images = sorted(os.listdir(orient_path))

                        num_files = len(images)//120 # Number of volumes to extract

                        for n in range(num_files):

                            file = [f for f in images if ('_' + str(1 + 120*n) + '.')]

                            rep = '_' + str(n)

                            image_path = orient_path + file[0]

                            dcm = pydicom.read_file(image_path)

                            venc = dcm[0x2001101A].value[0]

                            vencs.append(venc)

                            files.append(patientID + '_' + orient + rep)


        # Check that destination path exists, otherwise, create it

        if not(os.path.isdir(dest_path)):

            os.makedirs(dest_path)

        np.savetxt(dest_path + study + '_venc_values.txt', np.array(vencs))

        np.savetxt(dest_path + study + '_venc_files.txt', np.array(files), fmt = '%s')

    else:
        
        print('Non-existing initial path. Please provide a valid path')


vencFromDCM(key, study, init_path, dest_path)    

In [43]:
studies = ['CKD2', 'hero', 'extr']

init_paths = ['/home/andres/Documents/_Data/CKD_Part2/2_QFlow_Measurements/',
             '/home/andres/Documents/_Data/Heroic/', '/home/andres/Documents/_Data/Extra/']

dest_path = '/home/andres/Documents/_Data/venc_info/'


def mat_loader(path, filename):
        
    """
    Load .mat file from Segment software

    Params:
        - path and filename

    Returns:
        - mat: dictionary with .mat file fields
        - st: structure field with data

    """

    #print('Conversion from .mat into array\n')

    mat = scipy.io.loadmat(path + filename) 

    st = mat.get('setstruct') 

    return mat, st


def repetitionExtractor(folder):
        
    """
    Extract whether files to be read are repeated acquisitions or not.

    Params:
        - inherited from class (see description at beginning)
        - folder: folder name where to check for information

    Return: repetition string (rep) ('_0' for first acquisition, '_1' for second... and so on)

    """ 

    for i in range(6):

        test_str = '_' + str(i)

        if test_str in folder:

            rep = '_' + str(i)

            break

        else:

            rep = '_0'

    return rep


def vencFromMAT(studies,init_paths, dest_path):
    
    """
    Extracts velocity encoding information from MAT files.
    
    Params:
    
    - studies: list of studies from where images are extracted
    
    - init_paths: folder from where to start looking for the MAT files with the VENCs
    
    - dest_path: folder where to save TXT files with results
    
    Outputs:
    
    - Text file with list of files
    
    - Text file with list of VENCs acquired
    
    """
    
    vencs = []
    
    names = []
    
    for study, init_path in zip(studies, init_paths): # Go through all studies and initial paths
        
        if os.path.isdir(init_path):

            measure_paths = sorted(os.listdir(init_path))

            for measure in measure_paths:

                if 'measure' in measure: # Folders with MAT files 
                    
                    if not('Andres' in measure):
                        
                        j = 2
                    
                    
                    else:
                        
                        j = 1
                    
                    for i in range(j):

                        measure_path = init_path + measure + '/'

                        files = sorted(os.listdir(measure_path))

                        for file in files:

                            if '.mat' in file: # Access only MAT files

                                mat, struct = mat_loader(measure_path, file)

                                venc = struct[0,0][28][0,0]
                                
                                vencs.append(venc)


                                if study == 'CKD2':

                                    patientID = study + '_' + file[:11]

                                    if '^' in patientID:
                                        
                                        patientID = patientID.replace('^','_')

                                    orient = orientationExtractor(file)

                                    if 'Andres' in measure:
                                        
                                        rep = repetitionExtractor(file)
                                    
                                    else:
                                        
                                        rep = '_' + str(i)
                                    
                                    filename = patientID + '_' + orient + rep


                                else:

                                    filename = file[:-4]
                             
                                names.append(filename)
                
        
        else:
            
            print('{} is not a valid path. Please provide a valid path\n'.format(init_path))
    
    
    # Check that destination path exists, otherwise, create it

    if not(os.path.isdir(dest_path)):

        os.makedirs(dest_path)

    np.savetxt(dest_path + 'rest_venc_values.txt', np.array(vencs))

    np.savetxt(dest_path + 'rest_venc_files.txt', np.array(names), fmt = '%s')
    

vencFromMAT(studies,init_paths, dest_path)



In [6]:
path = '/home/andres/Documents/_Data/CKD_Part2/1_Raw_data/CKD019_MRI3_20190829/sQFLOW_BHdx/'

file = 'CKD019_MRI3_20190829_IM_1.dcm'

dcm = pydicom.read_file(path + file)

venc = dcm[0x2001101A].value

print(venc)

[70.0, 0.0, 0.0]
